## waste management

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# !pip install pandas

### Source Inventory Data: 
Contains information about stock levels at different source DCs, SKU batches, and their remaining shelf life.
### Destination Demand Data: 
Contains demand data for SKUs at different destination DCs along with per-unit prices.
### Transportation Cost Data: 
Includes per-unit transportation cost and transit time between source and destination DCs.

In [24]:
import pandas as pd

demand_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_destination_demand.csv') # insert the path of the csv file
demand_data = pd.read_csv(demand_file_path)

source_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_source_inventory.csv') # insert the path of the csv file
source_data = pd.read_csv(source_file_path)

transport_file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/transportation_costs.csv') # insert the path of the csv file
transport_data = pd.read_csv(transport_file_path)

In [25]:
#preview the csv file
demand_data.head()

,Destination_DC,SKU,Demand
0,DC_1,SKU_1,287
1,DC_1,SKU_2,316
2,DC_1,SKU_3,104
3,DC_1,SKU_4,66
4,DC_1,SKU_5,257


In [26]:
#preview the csv file
source_data.head()

,Source_DC,SKU,Batch_ID,AvailableStock,ShelfLifeDays,PerUnitPrice
0,DC_1,SKU_1,SKU_1_Batch_1,398,17,8.402283
1,DC_1,SKU_1,SKU_1_Batch_2,156,10,2.825307
2,DC_1,SKU_1,SKU_1_Batch_3,238,23,8.214560
3,DC_1,SKU_1,SKU_1_Batch_4,152,28,2.046000
4,DC_1,SKU_2,SKU_2_Batch_1,264,13,3.149458


In [27]:
#preview the csv file
transport_data.head()

,Source_DC,Destination_DC,SKU,PerUnitTransportationCost,TransitTimeDays
0,DC_1,DC_1,SKU_1,2.668404,1
1,DC_1,DC_1,SKU_2,3.206992,4
2,DC_1,DC_1,SKU_3,3.344718,5
3,DC_1,DC_1,SKU_4,3.020413,1
4,DC_1,DC_1,SKU_5,2.812345,9


In [ ]:
# !pip install langchain-community

In [28]:
from langchain_community.document_loaders import DirectoryLoader, CSVLoader
csv_folder_path = '/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder'
loader = DirectoryLoader(csv_folder_path, glob="**/*.csv",
                             loader_cls=CSVLoader, loader_kwargs={'encoding': 'utf-8'})

In [ ]:
# !pip install langchain-google-genai
# !pip install python-dotenv

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from dotenv import load_dotenv
import os

load_dotenv()

embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=os.environ['GEMINI_API_KEY'])
index_creator = VectorstoreIndexCreator(embedding=embedding_model)
docsearch = index_creator.from_loaders([loader])

/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/.venv/lib/python3.10/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [42]:
retriever = docsearch.vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 5})

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.environ['GEMINI_API_KEY'])

In [44]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer or the question is not related to the provided context, just say that you don't know, don't try to make up an answer.
Use 10 sentences maximum, add bullet points where applicable and keep the answer as reasonable as possible.

You will now answer the questions from the provided context. If the questions is not relevant, just say you don't know the answer.
```{context}```

Question: ```{question}```
Helpful Answer:"""

prompt = PromptTemplate.from_template(template=template)

In [45]:
question = "what is the demand of SKU_1 in DC_1"

In [46]:
docs = retriever.invoke(question)

In [47]:
docs

[Document(id='9471bd02-9e31-4b13-b91f-cb6715100146', metadata={'source': '/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_destination_demand.csv', 'row': 11}, page_content='Destination_DC: DC_1\nSKU: SKU_12\nDemand: 27'),
 Document(id='42ce949f-4b14-4e04-8351-76bb614bc1f7', metadata={'source': '/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_destination_demand.csv', 'row': 0}, page_content='Destination_DC: DC_1\nSKU: SKU_1\nDemand: 287'),
 Document(id='d6c27e0f-e3fc-44a4-bd1a-e5a39f2f1a5c', metadata={'source': '/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_destination_demand.csv', 'row': 250}, page_content='Destination_DC: DC_6\nSKU: SKU_1\nDemand: 41'),
 Document(id='0d212628-539a-4443-a0b9-89dcbad36f4c', metadata={'source': '/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/csv_folder/updated_destination_demand.csv', 'row': 15}, page_content='Destination_DC: DC_1\nSKU: SKU_16\nDemand: 51'),
 

In [48]:
relevant_text = ''
for doc in docs:
  relevant_text += doc.page_content
print(relevant_text)

Destination_DC: DC_1
SKU: SKU_12
Demand: 27Destination_DC: DC_1
SKU: SKU_1
Demand: 287Destination_DC: DC_6
SKU: SKU_1
Demand: 41Destination_DC: DC_1
SKU: SKU_16
Demand: 51Destination_DC: DC_1
SKU: SKU_14
Demand: 120


In [50]:
prompt_formatted_str: str = prompt.format(question=question, context=relevant_text)

In [51]:
prompt_formatted_str

"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer or the question is not related to the provided context, just say that you don't know, don't try to make up an answer.\nUse 10 sentences maximum, add bullet points where applicable and keep the answer as reasonable as possible.\n\nYou will now answer the questions from the provided context. If the questions is not relevant, just say you don't know the answer.\n```Destination_DC: DC_1\nSKU: SKU_12\nDemand: 27Destination_DC: DC_1\nSKU: SKU_1\nDemand: 287Destination_DC: DC_6\nSKU: SKU_1\nDemand: 41Destination_DC: DC_1\nSKU: SKU_16\nDemand: 51Destination_DC: DC_1\nSKU: SKU_14\nDemand: 120```\n\nQuestion: ```what is the demand of SKU_1 in DC_1```\nHelpful Answer:"

In [52]:
response = llm.invoke(prompt_formatted_str)

In [53]:
response.content

"The demand of SKU_1 in DC_1 is 287.\n\nHere's a breakdown of the information provided:\n\n*   Destination DC: DC_1, SKU: SKU_1, Demand: 287"